# Symmetric noise -- KITTY MDE -- DEPTH ESTIMATION

In [ ]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import shutil

import numpy as np
from numpy.random import default_rng

def spoil(init, nu):
    spoiled = np.arange(len(init))
    idx_shuffle = np.arange(len(init))
    n_shuffle = int(nu * len(init))
    rng = np.random.default_rng()
    rng.shuffle(idx_shuffle)
    idx_shuffle = idx_shuffle[:n_shuffle] 
    idx_to_shuffle = np.sort(idx_shuffle)
    count = 0
    for i in range(len(init)):
        if (count < n_shuffle) and (i == idx_to_shuffle[count]):
            spoiled[i] = idx_shuffle[count]
            count += 1
        print(i, ' --> ', spoiled[i])
    #print(spoiled)
    return np.array(spoiled)

def generate_noise_NOCLASSES(inpath, nu, outpath_prefix='_noise_'): #symmetric noise
    # generate new path
    split_path = inpath.split('/')
    inpath_prefix = split_path[0]
    inpath_suffix = inpath[len(split_path[0]):] 
    newpath = inpath_prefix + outpath_prefix + str(nu) + inpath_suffix
    if not exists(newpath):
        makedirs(newpath)
        print('Data with noise_coeff=', nu, 'will be placed in the folder:')
        print(newpath)
    # generate new labels
    initial_labels = [f for f in listdir(inpath) if isfile(join(inpath, f))]
    noisy_idx = spoil(initial_labels, nu)
    # save new data
    for i in range(len(initial_labels)):
        shutil.copy(inpath + initial_labels[i],
                    newpath + initial_labels[noisy_idx[i]])
    print('Data successfully saved!\n')

In [ ]:
# nu is the percentage of noisy masks
for nu in [0.4, 0.8]:
    inpath = 'dataset_kitti_MDE/train/masks/'   #<---- сюда пишем путь к папке, где лежат маски для Train
    generate_noise_NOCLASSES(inpath, nu)  
    inpath = 'dataset_kitti_MDE/val/masks/'     #<---- сюда пишем путь к папке, где лежат маски для Test
    generate_noise_NOCLASSES(inpath, nu)   

# Symmetric noise WITH CLASSES -- KITTY SEGMENTATION DATASET

In [ ]:
import os
from os import listdir, makedirs
from os.path import isfile, join, exists
import shutil

import numpy as np
from numpy.random import default_rng

def find_names_in_fold(prefix):
    class_names = np.sort(os.listdir(prefix))
    if '.DS_Store' in class_names:
        place_DS_Store = np.where(class_names == '.DS_Store')
        class_names = np.delete(class_names, place_DS_Store[0][0])
    list_names = np.sort(os.listdir(prefix + class_names[0])).tolist()
    for i, x in enumerate(list_names):
        list_names[i] = os.path.join(class_names[0],x)
    for i in class_names[1:]:
        list_names_onefold = np.sort(os.listdir(prefix + i)).tolist()
        for j, x in enumerate(list_names_onefold):
            list_names_onefold[j] = os.path.join(i, x)
        list_names.extend(list_names_onefold)
    return list_names

def spoil(init, nu):
    spoiled = np.arange(len(init))
    idx_shuffle = np.arange(len(init))
    n_shuffle = int(nu * len(init))
    rng = np.random.default_rng()
    rng.shuffle(idx_shuffle)
    idx_shuffle = idx_shuffle[:n_shuffle] 
    idx_to_shuffle = np.sort(idx_shuffle)
    count = 0
    for i in range(len(init)):
        if (count < n_shuffle) and (i == idx_to_shuffle[count]):
            spoiled[i] = idx_shuffle[count]
            count += 1
        print(i, ' --> ', spoiled[i])
    #print(spoiled)
    return np.array(spoiled)

def generate_noise(inpath, nu, outpath_prefix='_noise_'): #symmetric noise
    # generate new path
    split_path = inpath.split('/')
    inpath_prefix = split_path[0]
    inpath_suffix = inpath[len(split_path[0]):] 
    newpath = inpath_prefix + outpath_prefix + str(nu) + inpath_suffix
    if not exists(newpath):
        makedirs(newpath)
        print('Data with noise_coeff=', nu, 'will be placed in the folder:')
        print(newpath)
    # form initial labels and new folders
    class_names = np.sort(listdir(inpath))
    for class_name in class_names:
        #print(newpath + class_name)
        if not exists(newpath + class_name):
            makedirs(newpath + class_name)
            print(newpath + class_name)
            print('Created new path', newpath + class_name)
            #print(newpath)
    #print(newpath, class_names)
    # generate new labels
    initial_labels = find_names_in_fold(inpath) #[f for f in listdir(inpath) if isfile(join(inpath, f))]
    print(initial_labels)
    noisy_idx = spoil(initial_labels, nu)
    ## save new data
    for i in range(len(initial_labels)):
        shutil.copy(inpath + initial_labels[i],
                    newpath + initial_labels[noisy_idx[i]])
    print('Data successfully saved!\n')

In [ ]:
# nu is the percentage of noisy masks
for nu in [0.02, 0.05, 0.1, 0.2, 0.3, 0.4]:
    inpath = 'train_val_dataset/train/masks/' #<---- сюда пишем путь к папке, где лежат маски для MDE Train
    generate_noise(inpath, nu)  
    inpath = 'train_val_dataset/val/masks/'   #<---- сюда пишем путь к папке, где лежат маски для MDE Test
    generate_noise(inpath, nu)   

# Noise in IMAGE BOUDARY NO CLASSES -- LIPS DATASET 

In [ ]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import random 
import cv2 

from os import listdir, makedirs
from os.path import isfile, join, exists
import shutil

import numpy as np
from numpy.random import default_rng

def binarize(rocket, boudary):
    copy = np.array(rocket).copy()
    new_rocket = copy[:,:]
    
    new_rocket[new_rocket > boudary] = 255
    new_rocket[new_rocket <= boudary] = 0
    return new_rocket

def add_noise2lips(img): 
    print(img.shape)
    row , col, _ = img.shape 
    
    pts = []
    number_of_pixels = random.randint(30, 200) 
    i = 0
    while(i == 0): 
        y_coord=random.randint(0, row - 1) 
        x_coord=random.randint(0, col - 1)
        if (sum(img[y_coord][x_coord]) > 0):
            pts.append([x_coord, y_coord])
            i+=1
    while(i < number_of_pixels or sum(img[y_coord][x_coord]) > 0):
        y_coord += random.randint(-2, 2) 
        x_coord += random.randint(-2, 2)
        pts.append([x_coord, y_coord])
        i+=1
    print(pts)
    pts = np.array(pts)
    pts = pts.reshape((-1, 1, 2))  
    #print('here ', img[pts[0, 0, 0], pts[0, 0, 1]])
    color = (34, 255, 14)
    print(color)
    img = cv2.polylines(img, [pts], True, color, 10)
    return img 

def spoil_lip(in_filename, out_filename):
    img = cv2.imread(in_filename)
    #img = binarize(img, 1)
    img = add_noise2lips(img)
    cv2.imwrite(out_filename, img) 
    
def pick_lips(init, nu):
    spoiled = np.arange(len(init))
    initial = np.arange(len(init))
    idx_shuffle = np.arange(len(init))
    n_shuffle = int(nu * len(init))
    rng = np.random.default_rng()
    rng.shuffle(idx_shuffle)
    idx_shuffle = idx_shuffle[:n_shuffle] 
    idx_to_shuffle = np.sort(idx_shuffle)
    count = 0
    for i in range(len(init)):
        if (count < n_shuffle) and (i == idx_to_shuffle[count]):
            spoiled[i] = -1
            count += 1
        print(i, ' --> ', spoiled[i])
    #print(spoiled)
    return np.array(spoiled)
    
def generate_noise_lips_NOCLASSES(inpath, nu, outpath_prefix='_noise_'): #symmetric noise
    # generate new path
    split_path = inpath.split('/')
    inpath_prefix = split_path[0]
    inpath_suffix = inpath[len(split_path[0]):] 
    newpath = inpath_prefix + outpath_prefix + str(nu) + inpath_suffix
    if not exists(newpath):
        makedirs(newpath)
        print('Data with noise_coeff=', nu, 'will be placed in the folder:')
        print(newpath)
    # generate new labels
    initial_labels = [f for f in listdir(inpath) if isfile(join(inpath, f))]
    noisy_idx = pick_lips(initial_labels, nu)
    # save new data
    for i in range(len(initial_labels)):
        if noisy_idx[i] == -1:
            spoil_lip(inpath + initial_labels[i], newpath + initial_labels[i])
        else:
            shutil.copy(inpath + initial_labels[i], newpath + initial_labels[i])
    print('Data successfully saved!\n')

In [ ]:
# nu is the percentage of noisy masks
for nu in [0.02, 0.05, 0.1, 0.2, 0.3, 0.4]:
    inpath = 'initial_lips_masks_folder/'      #<---- сюда пишем путь к папке, где лежат маски для губ
    generate_noise_lips_NOCLASSES(inpath, nu)    

# Noise in IMAGE BOUDARY NO CLASSES -- TikTok Dansers DATASET

In [ ]:
pip install opencv-python

In [ ]:
import random 
import cv2 

from os import listdir, makedirs
from os.path import isfile, join, exists
import shutil

import numpy as np
from numpy.random import default_rng

def binarize(rocket, boudary):
    copy = np.array(rocket).copy()
    new_rocket = copy[:,:]
    
    new_rocket[new_rocket > boudary] = 255
    new_rocket[new_rocket <= boudary] = 0
    return new_rocket

def add_noise2lips(img): 
    print(img.shape)
    row , col, _ = img.shape 
    
    pts = []
    number_of_pixels = random.randint(30, 800) 
    i = 0
    while(i == 0): 
        y_coord=random.randint(0, row - 1) 
        x_coord=random.randint(0, col - 1)
        if (sum(img[y_coord][x_coord]) > 0):
            pts.append([x_coord, y_coord])
            i+=1
    while((i < number_of_pixels or sum(img[y_coord][x_coord]) > 0) and 3 < y_coord < row-3 and 3 < x_coord < col-3):
        y_coord += random.randint(-2, 2) 
        x_coord += random.randint(-2, 2)
        pts.append([x_coord, y_coord])
        i+=1
    #print(pts)
    pts = np.array(pts)
    pts = pts.reshape((-1, 1, 2))  
    #print('here ', img[pts[0, 0, 0], pts[0, 0, 1]])
    color = (0, 0, 0)
    #print(color)
    img = cv2.polylines(img, [pts], True, color, 15)
    
    pts = []
    number_of_pixels = random.randint(30, 800) 
    i = 0
    while(i == 0): 
        y_coord=random.randint(0, row - 1) 
        x_coord=random.randint(0, col - 1)
        if (sum(img[y_coord][x_coord]) == 0):
            pts.append([x_coord, y_coord])
            i+=1
    while((i < number_of_pixels or sum(img[y_coord][x_coord]) == 0) and 3 < y_coord < row-3 and 3 < x_coord < col-3):
        y_coord += random.randint(-2, 2) 
        x_coord += random.randint(-2, 2)
        pts.append([x_coord, y_coord])
        i+=1
    #print(pts)
    pts = np.array(pts)
    pts = pts.reshape((-1, 1, 2))  
    #print('here ', img[pts[0, 0, 0], pts[0, 0, 1]])
    color = (255, 255, 255)
    #print(color)
    img = cv2.polylines(img, [pts], True, color, 15)
    
    return img 

def spoil_lip(in_filename, out_filename):
    img = cv2.imread(in_filename)
    #img = binarize(img, 1)
    repeat = random.randint(1, 10)
    print(repeat)
    for _ in range(repeat):
        img = add_noise2lips(img)
    cv2.imwrite(out_filename, img) 
    
def pick_lips(init, nu):
    spoiled = np.arange(len(init))
    initial = np.arange(len(init))
    idx_shuffle = np.arange(len(init))
    n_shuffle = int(nu * len(init))
    rng = np.random.default_rng()
    rng.shuffle(idx_shuffle)
    idx_shuffle = idx_shuffle[:n_shuffle] 
    idx_to_shuffle = np.sort(idx_shuffle)
    count = 0
    for i in range(len(init)):
        if (count < n_shuffle) and (i == idx_to_shuffle[count]):
            spoiled[i] = -1
            count += 1
        #print(i, ' --> ', spoiled[i])
    #print(spoiled)
    return np.array(spoiled)
    
def generate_noise_lips_NOCLASSES(inpath, nu, outpath_prefix='_noise_'): #symmetric noise
    # generate new path
    split_path = inpath.split('/')
    inpath_prefix = split_path[0]
    inpath_suffix = inpath[len(split_path[0]):] 
    newpath = inpath_prefix + outpath_prefix + str(nu) + inpath_suffix
    if not exists(newpath):
        makedirs(newpath)
        print('Data with noise_coeff=', nu, 'will be placed in the folder:')
        print(newpath)
    # generate new labels
    initial_labels = [f for f in listdir(inpath) if isfile(join(inpath, f))]
    noisy_idx = pick_lips(initial_labels, nu)
    # save new data
    for i in range(len(initial_labels)):
        if noisy_idx[i] == -1:
            print(inpath + initial_labels[i])
            spoil_lip(inpath + initial_labels[i], newpath + initial_labels[i])
        else:
            shutil.copy(inpath + initial_labels[i], newpath + initial_labels[i])
    print('Data successfully saved!\n')

In [ ]:
# nu is the percentage of noisy masks
for nu in [0.4, 0.6, 0.8]:
    inpath = 'dataset_people/train/masks/' #<---- сюда пишем путь к папке, где лежат маски для губ
    generate_noise_lips_NOCLASSES(inpath, nu)     
    inpath = 'dataset_people/val/masks/' #<---- сюда пишем путь к папке, где лежат маски для губ
    generate_noise_lips_NOCLASSES(inpath, nu)     

# Noise in IMAGE BOUDARY NO CLASSES -- Lungs DATASET

In [ ]:
pip install opencv-python

In [ ]:
import random 
import cv2 

from os import listdir, makedirs
from os.path import isfile, join, exists
import shutil

import numpy as np
from numpy.random import default_rng

def binarize(rocket, boudary):
    copy = np.array(rocket).copy()
    new_rocket = copy[:,:]
    
    new_rocket[new_rocket > boudary] = 255
    new_rocket[new_rocket <= boudary] = 0
    return new_rocket

def add_noise2lips(img): 
    print(img.shape)
    row , col, _ = img.shape 
    
    pts = []
    number_of_pixels = random.randint(30, 150) 
    i = 0
    while(i == 0): 
        y_coord=random.randint(0, row - 1) 
        x_coord=random.randint(0, col - 1)
        if (sum(img[y_coord][x_coord]) > 0):
            pts.append([x_coord, y_coord])
            i+=1
    while((i < number_of_pixels or sum(img[y_coord][x_coord]) > 0) and 3 < y_coord < row-3 and 3 < x_coord < col-3):
        y_coord += random.randint(-2, 2) 
        x_coord += random.randint(-2, 2)
        pts.append([x_coord, y_coord])
        i+=1
    #print(pts)
    pts = np.array(pts)
    pts = pts.reshape((-1, 1, 2))  
    #print('here ', img[pts[0, 0, 0], pts[0, 0, 1]])
    color = (0, 0, 0)
    #print(color)
    thikness = random.randint(5, 15)
    img = cv2.polylines(img, [pts], True, color, thikness)
    
    pts = []
    number_of_pixels = random.randint(30, 150) 
    i = 0
    while(i == 0): 
        y_coord=random.randint(0, row - 1) 
        x_coord=random.randint(0, col - 1)
        if (sum(img[y_coord][x_coord]) == 0):
            pts.append([x_coord, y_coord])
            i+=1
    while((i < number_of_pixels or sum(img[y_coord][x_coord]) == 0) and 3 < y_coord < row-3 and 3 < x_coord < col-3):
        y_coord += random.randint(-2, 2) 
        x_coord += random.randint(-2, 2)
        pts.append([x_coord, y_coord])
        i+=1
    #print(pts)
    pts = np.array(pts)
    pts = pts.reshape((-1, 1, 2))  
    #print('here ', img[pts[0, 0, 0], pts[0, 0, 1]])
    color = (255, 255, 255)
    #print(color)
    thikness = random.randint(5, 15)
    img = cv2.polylines(img, [pts], True, color, thikness)
    
    return img 

def spoil_lip(in_filename, out_filename):
    img = cv2.imread(in_filename)
    #img = binarize(img, 1)
    repeat = random.randint(1, 10)
    print(repeat)
    for _ in range(repeat):
        img = add_noise2lips(img)
    cv2.imwrite(out_filename, img) 
    
def pick_lips(init, nu):
    spoiled = np.arange(len(init))
    initial = np.arange(len(init))
    idx_shuffle = np.arange(len(init))
    n_shuffle = int(nu * len(init))
    rng = np.random.default_rng()
    rng.shuffle(idx_shuffle)
    idx_shuffle = idx_shuffle[:n_shuffle] 
    idx_to_shuffle = np.sort(idx_shuffle)
    count = 0
    for i in range(len(init)):
        if (count < n_shuffle) and (i == idx_to_shuffle[count]):
            spoiled[i] = -1
            count += 1
        #print(i, ' --> ', spoiled[i])
    #print(spoiled)
    return np.array(spoiled)
    
def generate_noise_lips_NOCLASSES(inpath, nu, outpath_prefix='_noise_'): #symmetric noise
    # generate new path
    split_path = inpath.split('/')
    inpath_prefix = split_path[0]
    inpath_suffix = inpath[len(split_path[0]):] 
    newpath = inpath_prefix + outpath_prefix + str(nu) + inpath_suffix
    if not exists(newpath):
        makedirs(newpath)
        print('Data with noise_coeff=', nu, 'will be placed in the folder:')
        print(newpath)
    # generate new labels
    initial_labels = [f for f in listdir(inpath) if isfile(join(inpath, f))]
    noisy_idx = pick_lips(initial_labels, nu)
    # save new data
    for i in range(len(initial_labels)):
        if noisy_idx[i] == -1:
            print(inpath + initial_labels[i])
            spoil_lip(inpath + initial_labels[i], newpath + initial_labels[i])
        else:
            shutil.copy(inpath + initial_labels[i], newpath + initial_labels[i])
    print('Data successfully saved!\n')

In [ ]:
# nu is the percentage of noisy masks
for nu in [0.4, 0.6, 0.8]:
    inpath = 'dataset_med/train/masks/'  #<---- сюда пишем путь к папке, где лежат маски для губ
    generate_noise_lips_NOCLASSES(inpath, nu)     
    inpath = 'dataset_med/val/masks/'    #<---- сюда пишем путь к папке, где лежат маски для губ
    generate_noise_lips_NOCLASSES(inpath, nu) 